In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

from lexicon_use import form_text_vector
from estimators import estimate

from gsitk.preprocess import pprocess_twitter, simple, Preprocesser


moral_options = ('care', 'fairness', 'loyalty', 'authority', 'purity', 'non-moral')

Predict a single text.
In the following result, the output is the estimated probability of being relevant to either a vice or virtue of the corresponding moral trait.

In [9]:
# example from real tweet
text = '''
PLS help #HASHTAG's family. No one prepares for this. They are in need of any assistance you can offer
'''

estimate(text, moral='care', model='unigram+simon+freq')

array([0.99999993])

Any combination of the following models is valid:
* count
* freq
* unigram
* simon

For example, it is possible to use `simon+unigram`, `simon+unigram+count`, etc. Order does not affect the chosen model.

Also, it is possible to predict from a file, `lines.txt`. The format of this file is a text to analyze per line.

In [10]:
!cat lines.txt

My cat is happy
I really care for my cat
She hates going to the movies
PLS help #HASHTAG's family. No one prepares for this. They are in need of any assistance you can offer


In [13]:
estimate(None, moral='care', model='unigram+simon+freq', from_file=True)

array([1.03216899e-04, 1.28258061e-03, 2.10993531e-06, 9.99999926e-01])